# KI Challenge 
# Dokumentenklassifizierung
## Lena Hinkel, Lars Böcking, Mariana Zehender, Hongchen, Xiaoyu Yang

In [30]:
import random 
import pandas as pd
import time                #unsicher ob man das braucht 
import numpy as np
from pandas import DataFrame
import csv
from spacy.tokenizer import Tokenizer
from sklearn.model_selection import train_test_split

print ("Import of packages was successfull")


Import of packages was successfull


## Import Data - Negativ und Positiv gekennzeichnete Rezensionen für das Trainieren des Algorithmus 


In [2]:
with open("pos.txt") as file:
    dataPos = file.readlines()
with open("neg.txt") as file: 
    dataNeg = file.readlines()

## Vorbereitungen für NLP (Natural Language Processing) = Stemming and Lemmatiization

### Erstmal Preprocessing
#### (Aufbereiten und Analysieren von unstruktuierten Texten - Siehe Datensätze dataPos und dataNeg)

Lemmatisierung: Reduziere ein Wort – mittels eines festgelegten Korpus – auf den Wortstamm.
traf, trifft, treffen, treffe à treffen (aus Übung 1)

Wortfilterung (Stopwords removal: Lösche Stoppwörter, z.B. der, die, das, ein, über Stopplisten im Internet (aus Übung 1) - Lösche Zahlen und Satzzeichen --> Ersetze diese Zeichen mit Leerzeichen

Synonyme: Ersetze Wörter mit derselben Bedeutung mit einem identischen Synonym. Meist mittels lexikalischer Datenbank, wie bspw. Wordnet (aus Übung 1) 



In [3]:
# 1 Stopwords Removal

documentLen = len(dataPos)

intab = "!\"#$%&'()*+, -./:;<=>?@[\]^_`{|}~1234567890" 
outtab = ""
for i in range(len(intab)):
    outtab += " "

for i in range(documentLen): 
    dataPos[i] = dataPos[i].translate(str.maketrans(intab, outtab))
    dataNeg[i] = dataNeg[i].translate(str.maketrans(intab, outtab))
    dataPos[i]=dataPos[i].lower()
    dataNeg[i]=dataNeg[i].lower()

In [4]:
print("erster Versuch lemmatisierung")

"""# 2 Lemmatisierung 
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 
  
print(dataPos[0])

for i in range(documentLen):
    cache = ""
    for word in dataPos[i].split():
        cache += " " +lemmatizer.lemmatize(word)
    dataPos[i] = cache
    
print(dataPos[0])
"""

erster Versuch lemmatisierung


'# 2 Lemmatisierung \nimport nltk\nnltk.download(\'wordnet\')\nfrom nltk.stem import WordNetLemmatizer \n  \nlemmatizer = WordNetLemmatizer() \n  \nprint(dataPos[0])\n\nfor i in range(documentLen):\n    cache = ""\n    for word in dataPos[i].split():\n        cache += " " +lemmatizer.lemmatize(word)\n    dataPos[i] = cache\n    \nprint(dataPos[0])\n'

In [5]:
import spacy 
sp = spacy.load('en_core_web_sm', disable=["parser", "ner", "tagger"])
token = Tokenizer(sp.vocab)

print(dataPos[0])

for i in range(documentLen):
    dataPos[i] = token(dataPos[i])
    cache = ""
    for words in dataPos[i]:
         cache += " " +words.lemma_            
    dataPos[i] = cache
    dataNeg[i] = token(dataNeg[i])
    cache2 = ""
    for words in dataNeg[i]:
         cache2 += " " +words.lemma_            
    dataNeg[i] = cache2
    
print(dataPos[0])

this mascara really is all its cracked up to be and well worth the price  leaves my lashes looking fuller and curled  definitely is the best mascara i ve ever used 

 this mascara really be all its crack up to be and good worth the price   leave my lash look full and curl   definitely be the well mascara i have ever use 



In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer #tfidf steht für Term Frequency-Inverse Document Frequency


print(type(dataPos))
print(len(dataPos))
print(len(dataNeg))

dataCombined = []

for rec in dataPos:
    dataCombined.append(str(rec))
for rec in dataNeg:
    dataCombined.append(str(rec))

print(len(dataCombined))

corpus = dataCombined

cv = CountVectorizer()
word_count_vector = cv.fit_transform(corpus)

print(word_count_vector.shape)

vectorizer = TfidfVectorizer()
matrix = vectorizer.fit_transform(corpus)

<class 'list'>
2507
2500
5007
(5007, 10158)


In [45]:
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [46]:
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])

df_idf.sort_values(by=['idf_weights'])

,idf_weights
be,1.159423
and,1.194698
it,1.200294
this,1.206411
the,1.206411
...,...
sbsolutely,8.825645
fixture,8.825645
fixer,8.825645
saturation,8.825645


In [47]:
# count matrix
count_vector= cv.transform(dataCombined)
 
# tf-idf scores
tf_idf_vector= tfidf_transformer.transform(count_vector)

### Vektorisierung


In [50]:
feature_names = cv.get_feature_names()
 
#get tfidf vector for first document
first_document_vector=tf_idf_vector[2]
 
#print the scores
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False)

,tfidf
acne,0.271585
and,0.271069
system,0.229680
be,0.227193
that,0.183026
...,...
flay,0.000000
flayaways,0.000000
flea,0.000000
fleck,0.000000


In [56]:
print(len(dataCombined))
print(len(dataCombined[0]))
print(tf_idf_vector.shape)
print(tf_idf_vector[0].shape)

5007
157
(5007, 10158)
(1, 10158)


in "tf_idf_vector" haben wir alle 5007 reviews in Form der tf_idf abgespeichert. Das bedeutet, dass wir jedes Review auf einen Vektor der Größe 10158 abbilden 
hiermit wollen wir im nächsten schritt ein neuronales Netz trainieren 

In [71]:
print(type(tf_idf_vector))

<class 'scipy.sparse.csr.csr_matrix'>


In [88]:
x_train = tf_idf_vector.toarray()
print(len(x_train))
print(len(x_train[0]))

y_train = []

for i in range(len(dataPos)):
    y_train.append(1)
for i in range(len(dataNeg)):
    y_train.append(0)


5007
10158


In [89]:
import keras 

In [90]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.33, random_state=1)

In [112]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score

print('Logistic Regression')
clf = LogisticRegression()
clf.fit(X_train, Y_train)

predictions = clf.predict(X_test)

print(f'Accuracy: {round(accuracy_score(Y_test, predictions),4)}')


Logistic Regression
Accuracy: 0.8312


/Users/Mariana/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [109]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam

print(len(X_train))
input_shape = X_train[0].shape

model = Sequential()
model.add(Dense(10000, input_shape=input_shape))
model.add(Activation('relu'))

model.add(Dense(100, input_shape=input_shape))
model.add(Activation('relu'))

model.add(Dense(10, input_shape=input_shape))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('softmax'))
          
          
# Let's train the model using RMSprop
model.compile(loss='binary_crossentropy',
              optimizer="Adam",
              metrics=['accuracy'])

model.fit(X_train, Y_train,
              batch_size=1,
              epochs=10,
              validation_data=(X_test, Y_test),
              shuffle=True)
    
scores = model.evaluate(X_test, Y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

3354
Train on 3354 samples, validate on 1653 samples
Epoch 1/10
   9/3354 [..............................] - ETA: 2:09:16 - loss: 8.5185 - accuracy: 0.4444

KeyboardInterrupt: 